In [50]:
import re
from selenium import webdriver

In [51]:
def DisplayHowToUse():
    print('''
    >>> SETUP COMPLETE
    Usage Manual:
    play <search term> : Play the Video
    pause : Pauses the video
    unpause : Unpauses the video
    mute : Mutes the video
    unmute : Unmutes the video
    seek-to <time in seconds> : Seeks to the input time\n\n
    ''')

In [52]:
def LoginToFb(fb, email, password):
    email_field = fb.find_element_by_id('email')
    pass_field = fb.find_element_by_id('pass')
    email_field.send_keys(email)
    pass_field.send_keys(password)
    email_field.submit()

In [53]:
def MoveToMessageColumn(fb):
    profile_url=[a for a in fb.find_elements_by_tag_name('a') if a.get_attribute('title') == 'Profile'][0].get_attribute('href')
    re_search = re.search(r'(\?id=\d+)$', profile_url)

    FB_ID = ''

    if re_search:
        # Profiles with no username
        FB_ID = re_search.group(0)
        FB_ID = FB_ID.replace('?id=', '')
    else:
        # Profiles with username
        FB_ID = profile_url[profile_url.rfind('/') + 1:]

    fb.get('https://www.facebook.com/messages/t/' + FB_ID)

In [54]:
def ParseMessageAndExecute(message, fb, yt):
    message = message.lower()

    split_message = message.split()

    if re.search(r'^(play\s)', message):
        search_term = ' '.join(split_message[1:])
        search_box = yt.find_element_by_name('search_query')
        search_box.send_keys(search_term)
        search_box.submit()
        link = yt.find_elements_by_id('dismissable')[0].find_elements_by_tag_name('a')[0].get_attribute('href')
        yt.get(link)
    elif re.search(r'^(pause)$', message.strip()):
        yt.execute_script("document.getElementById('movie_player').pauseVideo();")
    elif re.search(r'^(unpause)$', message.strip()):
        yt.execute_script("document.getElementById('movie_player').playVideo();")
    elif re.search(r'^(mute)$', message.strip()):
        yt.execute_script("document.getElementById('movie_player').mute();")
    elif re.search(r'^(unmute)$', message.strip()):
        yt.execute_script("document.getElementById('movie_player').unMute();")
    elif re.search(r'^(seek-to)\s', message.strip()):
        time_value = split_message[1]
        yt.execute_script("document.getElementById('movie_player').seekTo(" + time_value + ",true);")
        

In [55]:
def ReadMessage(fb, yt):
    previous_message= (fb.find_elements_by_css_selector('span._3oh-._58nk'))[-1].text
    while True:
        fb.implicitly_wait(5)
        current_message= (fb.find_elements_by_css_selector('span._3oh-._58nk'))[-1].text
        if not current_message == previous_message:
            print(current_message)
            ParseMessageAndExecute(current_message, fb, yt)
            previous_message = current_message
        time.sleep(0.1)

In [56]:
if __name__ == '__main__':
    email = 'user_email'
    password = 'user_password'

    print('Setting Up ... Please Wait')

    chrome_opts = webdriver.ChromeOptions()
    chrome_opts.add_argument('--disable-notifications')

    fb= webdriver.Chrome(chrome_options=chrome_opts)
    yt= webdriver.Chrome()
    fb.get('https://www.facebook.com')
    yt.get('https://www.youtube.com')

    LoginToFb(fb, email, password)
    MoveToMessageColumn(fb)
    DisplayHowToUse()
    print('message log:')
    ReadMessage(fb, yt)

Setting Up ... Please Wait

    >>> SETUP COMPLETE
    Usage Manual:
    play <search term> : Play the Video
    pause : Pauses the video
    unpause : Unpauses the video
    mute : Mutes the video
    unmute : Unmutes the video
    seek-to <time in seconds> : Seeks to the input time


    
message log:
pause
play venom


WebDriverException: Message: chrome not reachable
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)
